In [15]:
import numpy as np
import sympy as sp

#### Constants (Peacemaker)

In [16]:
pi = 4.0 * np.arctan(1.0)
planck = 6.62606957e-34         # J s
avogadro = 6.0221413e23
kb = 1.3806488e-23              # J K^-1
speed_of_light = 299792458.0    # m s^-1
amu = 1.660538921e-27           # kg
gas_constant = avogadro*kb
hbar = planck/(2.0*pi)
global_eps = 1.0e-10

# Moment of inertia - process_coordinate_record : cluster.f90

The subroutine calculates the moments of inertia for a cluster.
For that, first the total mass of the cluster is calculated. 
Using the coordinates of the atoms each, the center of mass is determined and set as origin.
Then the inertia tensor is calculated.
This tensor is diagonalized to obtain the eigenvalues which correspond to the moments of inertia in x, y and z direction of the corresponding cluster.
The moments of inertia are saved, and the clusters are assigned as atom, linear or nonlinear

### Calculation of the total mass

### Diagonalization of symmetric 3x3 matrices (Direct calculation)

Starting with a symmetric 3x3 matrix.

$ \text{A} = \begin{pmatrix} a_{11} & a_{12} & a_{13} \\ a_{21} & a_{22} & a_{23} \\ a_{31} & a_{32} & a_{33} \end{pmatrix}$, </p>
$ \det(\text{A}) = a_{11}a_{22}a_{33} + 2a_{12}a_{23}a_{13} - a_{13}^2a_{22} - a_{23}^2a_{11} - a_{12}^2a_{33}$, </p>
$ \text{tr}(\text{A}^2) = a_{11}^2+a_{22}^2+a_{33}^2 + 2 (a_{12}^2+a_{23}^2+a_{13}^2)$, </p>
$ \text{tr}^2(\text{A}) = a_{11}^2+a_{22}^2+a_{33}^2 + 2 ( a_{11}a_{33}+ a_{11}a_{22}+ a_{22}a_{33})$ </p></br>

For the calculation of the eigenvalues, the characteristic polynomial is needed. </p>
$ \text{A} \nu = \alpha \nu $ </p>
$\small p_{\text{A}}(\alpha) = \det(\text{A} - \alpha I) = -\alpha^3 + \alpha^2 (a_{11}+a_{22}+a_{33}) + \alpha (a_{12}^2+a_{23}^2+a_{13}^2 - a_{11}a_{33}- a_{11}a_{22}- a_{22}a_{33}) + a_{11}a_{22}a_{33} + 2a_{12}a_{23}a_{13} - a_{13}^2a_{22} - a_{23}^2a_{11} - a_{12}^2a_{33}$ </p>
$ 0 = \alpha^3 - \text{tr}(\text{A}) \alpha^2 - \frac{1}{2} (\text{tr}(\text{A}^2)-\text{tr}^2(\text{A})) - \det(\text{A})$ </p> </br>
    
This is solved via the trigonometric solution. </p>
$ \text{B} \mu = \beta \mu $ </p>
$ \text{A} = p \text{B} + q \text{I}~~~~$ with I being the identity matrix.</p></br>
A and B have the same eigenvalues if: </p>
$ \alpha = p \beta + q$ </p></br>
$ q = \text{tr}(\text{A})/3~~~~$ and $~~~~p = \sqrt{\frac{\text{tr}(\text{A}-q\text{I})^2}{6}}$ </p> 
$ \det(\beta \text{I} - B) = \beta^3 -3\beta -\det(B)$ </p></br>

Substitution and simplification:</p>
$\beta = 2\cos(\theta)$ </p>
$cos(3\theta) = 4\cos^3(\theta) - 3\cos(\theta)$ </p>
Leads to: </p>
$ \cos(3\theta) = \det(B)/2 $ </p></br>

Thus the eigenvalues are: </p>
$ \beta = 2 \cos(\frac{1}{3} \arccos(\det(B)/2)+\frac{2k\pi}{3}) $, $~~~~k = 0,1,2$


In [17]:
# Calculates the eigenvalues of a 3x3 matrix
def diagonalization(a, eig):
   # a     : input     - 3x3 matrix
   # eig   : output    - array of the three eigenvalues
    
   p1 = a[0][1]**2 + a[0][2]**2 + a[1][2]**2
   if (p1 == 0):
      # The matrix is already in its diagonal form
      eig = np.diag(a)
   else:
      q = np.trace(a)/3
      p2 = (a[0][0]-q)**2 + (a[1][1]-q)**2 + (a[2][2]-q)**2 + 2*p1
      p = np.sqrt(p2/6)
      b = (a - q*np.identity(3))/p
      r = np.linalg.det(b)/2 # determinant
      print(r)

   # In exact arithmetic for a symmetric matrix  -1 <= r <= 1
   # but computation error can leave it slightly outside this range.
   if (r <= -1):
      phi = pi/3
   elif (r >= 1):
      phi = 0
   else:
      phi = np.arccos(r)/3

   # Calculation of the eigenvalues
      
   # the eigenvalues satisfy eig3 <= eig2 <= eig1
   eig[0] = q + 2 * p * np.cos(phi)
   eig[2] = q + 2 * p * np.cos(phi + (2*pi/3))
   eig[1] = 3 * q - eig[0] - eig[2]     # since trace(A) = eig1 + eig2 + eig3

   return(eig)


#### Check with an external python module

In [18]:
def diagonalization_sympy(a, eig):
    
   a = sp.Matrix(a)
   eig = a.eigenvals()
   
   # sort eigenvalues from biggest to smallest
   eig = sorted(eig, reverse=True)

   return(eig)

#### Tests

Test matrix </p>
$ \text{A} = \begin{pmatrix} 2 & 2 & -1 \\ 2 & 1 & 0 \\ -1 & 0 & 4 \end{pmatrix}~~~~~~~~~~~~~$ Eigenvalues: $2 - \sqrt{7},~~ 3,~~ 2 + \sqrt{7}$

In [19]:
# Matrix and initial eigenvalues
a = [[2,2,-1],[2,1,0],[-1,0,4]]
eig = np.zeros(3)

# Calculation with direct method
direct = diagonalization(a, eig)
print('Direct method : ',direct)

# Calculation with sympy
sympy = diagonalization_sympy(a, eig)
print('sympy         : ',sympy)
print()

# Check wether differences are smaller than 10^-10
diff = np.abs(direct - sympy)
for i in range(3):
    if (diff[i] < 1.0e-10):
        print('Eigenvalue ',i+1,' is the same')
    else:
        print('Eigenvalue ',i+1,' is different')

-0.6008383824567203
Direct method :  [ 4.64575131  3.         -0.64575131]
sympy         :  [2 + sqrt(7), 3, 2 - sqrt(7)]

Eigenvalue  1  is the same
Eigenvalue  2  is the same
Eigenvalue  3  is the same
